### Implementation on VGGNet

이제 이를 가지고 좀 더 복잡한 모델에 적용해보고자 합니다.
http://cs231n.github.io/convolutional-networks/#case 에 있는 VGGNet을 구현해보겠습니다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 100
display_step = 200

# Network parameters
n_input = 224*224*3
n_classes = 1000
dropout = 0.5

# Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # to keep dropout probability

In [2]:
def conv2d_simple(x, filter_size, dim_in, dim_out, name, strides=1):
    with tf.variable_scope(name) as scope:
        if type(filter_size)==int:
            filter_height=filter_size
            filter_width=filter_size
        elif type(filter_size)==list:
            filter_height,filter_width=filter_size
        w = tf.get_variable('w', shape=[filter_height, filter_width, dim_in, dim_out],
                initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
            
        b = tf.get_variable('b', shape=[dim_out])
        x = tf.nn.conv2d(x,w,strides=[1,strides,strides,1],padding='SAME')
        x = tf.nn.bias_add(x,b)
    print "After convolution: ", x.get_shape().as_list()
    return tf.nn.relu(x)

In [3]:
def fc_simple(x,dim_out,name,is_last=False):
    with tf.variable_scope(name) as scope:
        size_arr = x.get_shape().as_list()
        dim_in = np.prod(np.asarray(size_arr[1:]))
        x = tf.reshape(x, [-1,dim_in])
        dim_in = x.get_shape().as_list()[1]
        wd = tf.get_variable('wd', shape=[dim_in, dim_out],
                initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        bd = tf.get_variable('bd', shape=[dim_out])
        fc = tf.add(tf.matmul(x, wd),bd)
        if not is_last:
            fc = tf.nn.relu(fc)
            print "Fully connected:   ", fc.get_shape().as_list()
            return fc
        print "Output:            ", fc.get_shape().as_list()
        return fc

In [4]:
def maxpool2d(x, k=2):
    pool=tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')
    print "After max_pooling: ", pool.get_shape().as_list()
    return pool

In [5]:
def VGGNet_simple(x, dropout):
    with tf.variable_scope('VGGNet_simple'):
        # Reshape input picture
        x = tf.reshape(x, shape=[-1, 224, 224, 3])
        print "x shape", x.get_shape().as_list()

        # 2 convolutions and 1 max-pooling
        conv1 = conv2d_simple(x, 3, 3, 64, 'conv1')
        conv2 = conv2d_simple(conv1, 3, 64, 64, 'conv2')
        pool1 = maxpool2d(conv2, k=2)

        # 2 convolutions and 1 max-pooling
        conv3 = conv2d_simple(pool1, 3, 64, 128, 'conv3')
        conv4 = conv2d_simple(conv3, 3, 128, 128, 'conv4')
        pool2 = maxpool2d(conv4, k=2)

        # 3 convolutions and 1 max-pooling
        conv5 = conv2d_simple(pool2, 3, 128, 256, 'conv5')
        conv6 = conv2d_simple(conv5, 3, 256, 256, 'conv6')
        conv7 = conv2d_simple(conv6, 3, 256, 256, 'conv7')
        pool3 = maxpool2d(conv7, k=2)

        # 3 convolutions and 1 max-pooling
        conv8 = conv2d_simple(pool3, 3, 256, 512, 'conv8')
        conv9 = conv2d_simple(conv8, 3, 512, 512, 'conv9')
        conv10 = conv2d_simple(conv9, 3, 512, 512, 'conv10')
        pool4 = maxpool2d(conv10, k=2)

        # 3 convolutions and 1 max-pooling
        conv11 = conv2d_simple(pool4, 3, 512, 512, 'conv11')
        conv12 = conv2d_simple(conv11, 3, 512, 512, 'conv12')
        conv13 = conv2d_simple(conv12, 3, 512, 512, 'conv13')
        pool5 = maxpool2d(conv13, k=2)

        # Fully connected layers
        fc1 = fc_simple(pool5, 4096, 'fc1')
        fc2 = fc_simple(fc1, 4096, 'fc2')
        out = fc_simple(fc2, n_classes, 'out',is_last=True)
        return out

In [6]:
#tf.get_variable_scope().reuse_variables()
# Construct model
pred = VGGNet_simple(x, keep_prob)

x shape [None, 224, 224, 3]
After convolution:  [None, 224, 224, 64]
After convolution:  [None, 224, 224, 64]
After max_pooling:  [None, 112, 112, 64]
After convolution:  [None, 112, 112, 128]
After convolution:  [None, 112, 112, 128]
After max_pooling:  [None, 56, 56, 128]
After convolution:  [None, 56, 56, 256]
After convolution:  [None, 56, 56, 256]
After convolution:  [None, 56, 56, 256]
After max_pooling:  [None, 28, 28, 256]
After convolution:  [None, 28, 28, 512]
After convolution:  [None, 28, 28, 512]
After convolution:  [None, 28, 28, 512]
After max_pooling:  [None, 14, 14, 512]
After convolution:  [None, 14, 14, 512]
After convolution:  [None, 14, 14, 512]
After convolution:  [None, 14, 14, 512]
After max_pooling:  [None, 7, 7, 512]
Fully connected:    [None, 4096]
Fully connected:    [None, 4096]
Output:             [None, 1000]


In [7]:
# Define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
# Define optimizer
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
# Find correct prediction
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
# Get accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# initialize variables
init = tf.initialize_all_variables()
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.733)
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True,
                        gpu_options=gpu_options)

In [8]:
# apply config when starting session
sess = tf.InteractiveSession(config=config)
init.run()